In [2]:
import pandas as pd
import requests
import numpy as np
import statsapi

In [3]:
url = 'http://statsapi.mlb.com/api/v1/schedule/games/?sportId=1'
r = requests.get(url=url).json()
dates = r['dates'][0]['games']
df = pd.DataFrame(dates)
# df

In [5]:
HOST = 'https://statsapi.mlb.com'
url = HOST + df.loc[0,'link']
r = requests.get(url=url).json()
player_df = pd.DataFrame(r['gameData']['players'])
player_df_all = pd.DataFrame(['players'])
player_df = player_df.T
gameteams_df = pd.DataFrame(r['gameData']['teams'])
# player_df.T

In [ ]:
# The cell below holds all of the functions needed to get the final 'dayScore' of each player in the subsequent cell

In [11]:
# Returns 'hits per 9 innings' stat for given pitcher
def get_HP9(player_id):
    player = statsapi.player_stat_data(int(player_id), group= "pitching", type="season", sportId=1)
    if len(player['stats']) == 0:
        return 15
    return float(player['stats'][0]['stats']['hitsPer9Inn'])

# Fills list of away and home players with the players of each team
def get_player_teams(player_df, away_id, away_players, home_players):
    away_roster = statsapi.roster(away_id)
    for player in player_df['fullName']:
        if away_roster.find(player) != -1:
            away_players.append(player)
        else:
            home_players.append(player)

# Returns 'batting average' stat for given player
def get_avg(player_id):
    player = statsapi.player_stat_data(int(player_id), group= "hitting", type="season", sportId=1)
    if player['position'] == 'P':
        return 0
    return float(player['stats'][0]['stats']['avg'])

# Returns 'at bats' stat for given player
def get_atBats(player_id):
    player = statsapi.player_stat_data(int(player_id), group= "hitting", type="season", sportId=1)
    if player['position'] == 'P':
        return 0
    return float(player['stats'][0]['stats']['atBats'])

In [ ]:
# The cell below traverses through each game being played today

In [19]:
# Empty list to hold all game dfs
all_player_list = []

# Traverses through each game being played today
for game in range (0, len(df)):
    # Gets the necessary general info for each game (teams, players, etc.)
    url = HOST + df.loc[game,'link']
    r = requests.get(url=url).json()
    s = HOST + '/api/v1/schedule?sportId=1&hydrate=probablePitcher&startDate=2024-07-23&endDate=2024-07-23'
    t = requests.get(url=s).json()
    t_df = pd.DataFrame(t['dates'][0]['games'])
    gameteams_df = pd.DataFrame(r['gameData']['teams'])
    # player_df set equal to all the players for the given game
    player_df = pd.DataFrame(r['gameData']['players'])
    player_df = player_df.T

    # Assigns 'batting avg' stat to every player in player_df
    averages = player_df.id.apply(get_avg)
    player_df = player_df.assign(avg=averages.values)
    # Assigns 'at bats' stat to every player in player_df
    player_atBats = player_df.id.apply(get_atBats)
    player_df = player_df.assign(atBats=player_atBats.values)
    # Assigns HP9 stat of opposing pitcher to every player in player_df (initially set to 0 here)
    oppHP9 = player_df.id * 0
    player_df = player_df.assign(oppHP9=oppHP9.values)
    

    # Gets team id of away team
    away_id = int(df.T[game]['teams']['away']['team']['id'])

    # Creates empty list to hold (the full names of) the players of each team
    away_players = []
    home_players = []

    # Fills the player list of away and home teams
    get_player_teams(player_df, away_id, away_players, home_players)

    # Gets the HP9 of the away team starting pitcher
    away_pitcher_id = t_df['teams'][game]['away']['probablePitcher']['id']
    away_HP9 = float(get_HP9(away_pitcher_id))
    # Gets the HP9 of the home team starting pitcher
    home_pitcher_id = t_df['teams'][game]['home']['probablePitcher']['id']
    home_HP9 = float(get_HP9(home_pitcher_id))

    # Assigns HP9 stat of opposing pitcher to every player in player_df
    for player_name in away_players:
        player_df.loc[player_df['fullName'] == player_name, 'oppHP9'] = home_HP9
    for player_name in home_players:
        player_df.loc[player_df['fullName'] == player_name, 'oppHP9'] = away_HP9

    # Adds player_df to the list of all game dfs
    all_player_list.append(player_df)


# Combines all of the game dfs into one df with every player playing today
all_player_df = pd.concat(all_player_list)

# all_player_df.T


In [26]:
# Gets the 'batScore' (hitter's hitting score) stat and assigns it to every player
batScore = all_player_df.avg * all_player_df.atBats / 30
all_player_df = all_player_df.assign(batScore=batScore.values)

# Gets the 'batScore' (hitter's matchup score for today) stat and assigns it to every player
dayScore = all_player_df.batScore + all_player_df.oppHP9
all_player_df = all_player_df.assign(dayScore=dayScore.values)

all_player_df

,id,fullName,link,firstName,lastName,primaryNumber,birthDate,currentAge,birthCity,birthStateProvince,...,strikeZoneTop,strikeZoneBottom,nickName,nameTitle,nameSuffix,avg,atBats,oppHP9,batScore,dayScore
ID669134,669134,Luis Campusano,/api/v1/people/669134,Luis,Campusano,12,1998-09-29,25,Augusta,GA,...,3.42,1.57,NaN,NaN,NaN,0.233,219.0,9.55,1.700900,11.2509
ID673513,673513,Yuki Matsui,/api/v1/people/673513,Yuki,Matsui,1,1995-10-30,28,Yokohama,NaN,...,3.26,1.481,NaN,NaN,NaN,0.000,0.0,9.55,0.000000,9.55
ID650633,650633,Michael King,/api/v1/people/650633,Michael,King,34,1995-05-25,29,Rochester,NY,...,3.49,1.601,NaN,NaN,NaN,0.000,0.0,9.55,0.000000,9.55
ID678606,678606,Jose Ferrer,/api/v1/people/678606,Jose,Ferrer,47,2000-03-03,24,Maimon,NaN,...,3.411,1.565,NaN,NaN,NaN,0.000,0.0,6.09,0.000000,6.09
ID683083,683083,Nasim Nuñez,/api/v1/people/683083,Nasim,Nuñez,26,2000-08-18,23,Bronx,NY,...,3.29,1.49,NaN,NaN,NaN,0.143,14.0,6.09,0.066733,6.156733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID681351,681351,Logan O'Hoppe,/api/v1/people/681351,Logan,O'Hoppe,14,2000-02-09,24,West Islip,NY,...,3.34,1.52,NaN,NaN,NaN,0.276,304.0,6.71,2.796800,9.5068
ID668709,668709,JJ Bleday,/api/v1/people/668709,Jeffrey,Bleday,33,1997-11-10,26,Danville,PA,...,3.21,1.59,NaN,NaN,NaN,0.236,373.0,9.1,2.934267,12.034267
ID472610,472610,Luis García,/api/v1/people/472610,Luis,García,66,1987-01-30,37,Santo Domingo,NaN,...,3.467,1.589,Amadito,NaN,NaN,0.000,0.0,6.71,0.000000,6.71
ID542881,542881,Tyler Anderson,/api/v1/people/542881,Tyler,Anderson,31,1989-12-30,34,Las Vegas,NV,...,3.56,1.68,Mr. Duck,NaN,NaN,0.000,0.0,6.71,0.000000,6.71


In [30]:
valid_picks = all_player_df['dayScore'] > 14
top_picks = all_player_df.loc[valid_picks]
top_picks.sort_values(by = ['dayScore'], ascending = False)

,id,fullName,link,firstName,lastName,primaryNumber,birthDate,currentAge,birthCity,birthStateProvince,...,strikeZoneTop,strikeZoneBottom,nickName,nameTitle,nameSuffix,avg,atBats,oppHP9,batScore,dayScore
ID660271,660271,Shohei Ohtani,/api/v1/people/660271,Shohei,Ohtani,17,1994-07-05,30,Oshu,NaN,...,3.47,1.57,Showtime,NaN,NaN,0.310,393.0,10.68,4.061000,14.741
ID543760,543760,Marcus Semien,/api/v1/people/543760,Marcus,Semien,2,1990-09-17,33,San Francisco,CA,...,3.32,1.63,Simmy,NaN,NaN,0.244,406.0,11.35,3.302133,14.652133
ID608369,608369,Corey Seager,/api/v1/people/608369,Corey,Seager,5,1994-04-27,30,Charlotte,NC,...,3.5,1.49,Seags,NaN,NaN,0.269,334.0,11.35,2.994867,14.344867
ID669701,669701,Josh Smith,/api/v1/people/669701,Josh,Smith,8,1997-08-07,26,Baton Rouge,LA,...,3.19,1.48,NaN,NaN,NaN,0.285,312.0,11.35,2.964000,14.314
ID518692,518692,Freddie Freeman,/api/v1/people/518692,Frederick,Freeman,5,1989-09-12,34,Villa Park,CA,...,3.7,1.82,Phillip,NaN,NaN,0.288,375.0,10.68,3.600000,14.28
ID606192,606192,Teoscar Hernández,/api/v1/people/606192,Teoscar,Hernández,37,1992-10-15,31,Cotui,NaN,...,3.55,1.63,Mr. Seeds,NaN,NaN,0.267,386.0,10.68,3.435400,14.1154
